# Querying an ML Model on an Amazon SageMaker AI Endpoint with an MCP Server hosted on Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host MCP (Model Context Protocol) servers on Amazon Bedrock AgentCore Runtime. We will use the Amazon Bedrock AgentCore Python SDK to wrap MCP tools as an MCP server compatible with Amazon Bedrock AgentCore.

The Amazon Bedrock AgentCore Python SDK handles the MCP server implementation details so you can focus on your tools' core functionality. It transforms your code into the AgentCore standardized MCP protocol contracts for direct communication.

In this example, we will create an MCP Server

### Tutorial Key Features

* Creating MCP servers that communicates with the SageMaker AI Endpoint
* Testing MCP servers locally
* Hosting MCP servers on Amazon Bedrock AgentCore Runtime
* Invoking deployed MCP servers with authentication

In [ ]:
%pip install-r requirements.txt

In [ ]:
%store -r

## Understanding MCP (Model Context Protocol)

MCP is a protocol that allows AI models to securely access external data and tools. Key concepts:

* **Tools**: Functions that the AI can call to perform actions
* **Streamable HTTP**: Transport protocol used by AgentCore Runtime
* **Session Isolation**: Each client gets isolated sessions via `Mcp-Session-Id` header
* **Stateless Operation**: Servers must support stateless operation for scalability

AgentCore Runtime expects MCP servers to be hosted on `0.0.0.0:8000/mcp` as the default path.

### Project Structure

Let's set up our project with the proper structure:

```
mcp_server_project/
├── mcp_server.py              # Main MCP server code
├── my_mcp_client.py          # Local testing client
├── my_mcp_client_remote.py   # Remote testing client
├── requirements.txt          # Dependencies
└── __init__.py              # Python package marker
```

## Creating MCP Server

Now let's create our MCP server with three simple tools. The server uses FastMCP with `stateless_http=True` which is required for AgentCore Runtime compatibility.

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse
import json
import boto3
import numpy as np

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def demand_forecasting_with_ml(sample_data):
	"""
	This tool predicts future demand based on historical data using a machine learning model.
	"""
	sagemaker_runtime = boto3.client("sagemaker-runtime")
	endpoint_name = "ml-models-as-tools"
	response = sagemaker_runtime.invoke_endpoint(
		EndpointName=endpoint_name,
		Body=json.dumps(sample_data),
		ContentType="application/json",
		Accept="application/json"
	)
	predictions = json.loads(response['Body'].read().decode("utf-8"))
	return np.array(predictions)

if __name__ == "__main__":
	mcp.run(transport="streamable-http")

### What This Code Does

* **FastMCP**: Creates an MCP server that can host your tools
* **@mcp.tool()**: Decorator that turns your Python functions into MCP tools
* **stateless_http=True**: Required for AgentCore Runtime compatibility
* **Tools**: Three simple tools demonstrating different types of operations

## Creating Local Testing Client

To test your MCP server locally:

1. **Terminal 1**: Start the MCP server
   ```bash
   python mcp_server.py
   ```
   
2. Run the agent in the cell below

In [ ]:
test_sample = [[6.0,
  9.0,
  18.0,
  2022.0,
  3.0,
  82.32242989519233,
  145.7780143415423,
  119.72411707626237,
  109.00943866750897,
  78.1850279953027,
  27.17729231087614,
  98.725083103897,
  32.3663008574393,
  114.30045179438677,
  31.154757059649896],
 [0.0,
  9.0,
  19.0,
  2022.0,
  3.0,
  112.29372828348961,
  92.05548651574294,
  149.53457153115409,
  72.54645882732314,
  78.59649696115221,
  27.4428007288024,
  94.82516865714953,
  28.874170960965433,
  115.04676214269898,
  30.378550374126004],
 [1.0,
  9.0,
  20.0,
  2022.0,
  3.0,
  94.93576927668948,
  97.7187873924136,
  106.5675777935643,
  133.46975024736352,
  77.82557137676875,
  26.886240262965835,
  93.80764940773277,
  28.67887997302126,
  113.67518074451117,
  30.4468958883678],
 [2.0,
  9.0,
  21.0,
  2022.0,
  3.0,
  92.32230830172934,
  34.350356241745,
  112.97609292100049,
  99.21268633211206,
  80.06729855054992,
  23.0356600622431,
  89.31238894610509,
  30.32804828287802,
  112.03617274871455,
  32.50610622726534],
 [3.0,
  9.0,
  22.0,
  2022.0,
  3.0,
  50.04244645821317,
  77.85694616139921,
  81.35399000575227,
  125.84635801167887,
  82.91003059614825,
  23.927978474812306,
  90.48396612287904,
  30.31376940936954,
  111.09982983101149,
  32.49915610665346]]

In [ ]:
from strands import Agent
from strands.tools.mcp.mcp_client import MCPClient
from utils.cognito_utils import get_token
from utils.agent_utils import create_streamable_http_transport,get_full_tools_list 
import os

def run_agent(mcp_url: str, access_token: str):
    mcp_client = MCPClient(lambda: create_streamable_http_transport(mcp_url, access_token))
     
    with mcp_client:
        tools = get_full_tools_list(mcp_client)
        print(f"Found the following tools: {[tool.tool_name for tool in tools]}")

        # Create an agent with these tools
        agent = Agent(
            model="us.anthropic.claude-3-5-haiku-20241022-v1:0", tools=tools
        )
        
        # Invoke the agent
        agent(
            f"These are the current demand values:\n\n<input>{test_sample}</input>\n\n"
            "Predict demand, please. Provide the output in JSON format {'predictions':<predictions>}."
            "Only reply with the prediction JSON, nothing else. If the tool fails, just tell me the error."
        )
        
# run_agent(<MCP URL>, <Access token>)
run_agent("http://localhost:8000/mcp", None)

## Setting up Amazon Cognito for Authentication

AgentCore Runtime requires authentication. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed MCP server.

In [ ]:
allowed_clients = gateway["authorizerConfiguration"]["customJWTAuthorizer"]["allowedClients"]
discovery_url = gateway["authorizerConfiguration"]["customJWTAuthorizer"]["discoveryUrl"]
role_arn = gateway["roleArn"]

## Configuring AgentCore Runtime Deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

tool_name = "endpoint_via_runtime"

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": allowed_clients,
        "discoveryUrl": discovery_url,
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    agent_name=tool_name,
    entrypoint="mcp_server.py",
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    execution_role=role_arn,
    authorizer_configuration=auth_config,
    protocol="MCP",
)
print("Configuration completed ✓")

## Launching MCP Server to AgentCore Runtime

Now that we've got a docker file, let's launch the MCP server to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

## Checking AgentCore Runtime Status

Now that we've deployed the AgentCore Runtime, let's check for its deployment status and wait for it to be ready:

In [ ]:
print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

## Storing Configuration for Remote Access

Before we can invoke our deployed MCP server, let's store the Agent ARN and Cognito configuration in AWS Systems Manager Parameter Store and AWS Secrets Manager for easy retrieval:

In [ ]:
server_runtime_arn = launch_result.agent_arn
encoded_arn = server_runtime_arn.replace(':', '%3A').replace('/', '%2F')
mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
mcp_url

## Test the Agent with the remote MCP Server

Now let's create a client to test our deployed MCP server. This client will retrieve the necessary credentials from AWS and connect to the deployed server:

In [ ]:
from strands import Agent
from strands.tools.mcp.mcp_client import MCPClient
from utils.cognito_utils import get_token
from utils.agent_utils import create_streamable_http_transport,get_full_tools_list 
import os

def run_agent(mcp_url: str, access_token: str):
    mcp_client = MCPClient(lambda: create_streamable_http_transport(mcp_url, access_token))
     
    with mcp_client:
        tools = get_full_tools_list(mcp_client)
        print(f"Found the following tools: {[tool.tool_name for tool in tools]}")

        # Create an agent with these tools
        agent = Agent(
            model="us.anthropic.claude-3-5-haiku-20241022-v1:0", tools=tools
        )
        
        # Invoke the agent
        agent(
            f"These are the current demand values:\n\n<input>{test_sample}</input>\n\n"
            "Predict demand, please. Provide the output in JSON format {'predictions':<predictions>}."
            "Only reply with the prediction JSON, nothing else. If the tool fails, just tell me the error."
        )
        
# run_agent(<MCP URL>, <Access token>)
run_agent(mcp_url, get_token(gateway["gatewayId"]))

# 🎉 Congratulations!

You have successfully:

✅ **Created an MCP server** with custom tools  
✅ **Tested locally** using MCP client  
✅ **Set up authentication** with Amazon Cognito  
✅ **Deployed to AWS** using AgentCore Runtime  
✅ **Invoked remotely** with proper authentication  
✅ **Learned MCP concepts** and best practices  

Your MCP server is now running on Amazon Bedrock AgentCore Runtime and ready for production use!

## Summary

In this tutorial, you learned how to:
- Build MCP servers using FastMCP
- Configure stateless HTTP transport for AgentCore compatibility
- Set up JWT authentication with Amazon Cognito
- Deploy and manage MCP servers on AWS
- Test both locally and remotely
- Use MCP clients for tool invocation

The deployed MCP server can now be integrated into larger AI applications and workflows!